In [8]:

import pandas as pd


from typing import List, Dict, Tuple
import random
import pickle
import pprint

from surprise import KNNBasic, SVD, SVDpp, NMF
from surprise import Dataset, Reader

product_data = {
    "data_context": "books",
    "product_filepath": "data/products_books_v1_10_10.csv",
    "transactions_filepath": "data/transactions_books_v1_10_10.csv",
    "features": ["product_title", "product_image", "product_soup", "product_images"],
    "version": "1.0",
    "unique_name": "_books_v1_10_10",
}

print("looking at", "../" + product_data["product_filepath"])

productdf =  pd.read_csv("../" + product_data["product_filepath"])
transactiondf = pd.read_csv("../" + product_data["transactions_filepath"])


print(len(transactiondf))
productdf.head()
transactiondf.head()


looking at ../data/products_books_v1_10_10.csv
381082


,id,user_id,product_id,rate
0,e84cbad6-c5a7-49b1-9eb2-558a648998d6,276847,0446364193,0
1,e84cbad6-c5a7-49b1-9eb2-558a648998d6,276847,3379015180,0
2,e84cbad6-c5a7-49b1-9eb2-558a648998d6,276847,3404148576,4
3,e84cbad6-c5a7-49b1-9eb2-558a648998d6,276847,3423071516,5
4,e84cbad6-c5a7-49b1-9eb2-558a648998d6,276847,3442413508,5


In [9]:

class RecommendationAbstract():
    strategy_name: str = "REQUIRES IMPLEMENTATION"
    version: str = "REQUIRES IMPLEMENTATION"
    details: str = "REQUIRES IMPLEMENTATION"
    link: str = "REQUIRES IMPLEMENTATION"
    supports_single_recommendation: bool = "REQUIRES IMPLEMENTATION"
    supports_past_recommendation: bool = "REQUIRES IMPLEMENTATION"
    

    def __init__(self, products, product_data):
        self.products = products
        self.product_data = product_data
        self.model = None
        # populate id_to_products
        self.id_to_products = {}
        for product in self.products.to_dict(orient='records'):
            self.id_to_products[product['id']] = product

    def loadModel(self, model_code):
        """
        Load the model
        """
        self.model = model_code

    def train(self, verbose=False, transactions_train=None, users_train=None):
        """
        Train the model
        """
        # ... do training
        # self.model = trained_model
        
    def get_random_recommendation(self, n=1):
        """
        Get random recommendations
        """
        # Select n random rows from the DataFrame
        random_rows = self.products.sample(n)
        # Convert the selected rows to a list of dictionaries
        random_recommendations = random_rows.to_dict(orient='records')
        return random_recommendations



    def saveModel(self, model_code):
        """
        Save the model
        """
        # ... saves the model

    def id_to_productDetail(self, product_id: str) -> Dict[str, str]:
        """
        Return product details based on product id.
        """
        return self.id_to_products.get(product_id)

    def ids_to_products(self, ids: List[str]) -> List[Dict[str, str]]:
        """
        Return product details for a list of product ids.
        """
        return [self.id_to_productDetail(id) for id in ids]

    def like(self, keyword: str) -> List[str]:
        """
        Return a list of products that contain the given keyword in their title.
        """
        return [product for product in self.products if keyword in product['product_title']]

    def recommend_from_single(self, product_id: str, n=5) -> List[str]:
        """
        Return recommendations based on a single product.
        """
        target_name = self.id_to_productDetail(product_id)['product_title']
        keywords = target_name.split(" ")
        recommendations = []
        for keyword in keywords:
            recommendations.extend(self.like(keyword))
        
        random.shuffle(recommendations)
        return recommendations[:n]

    def recommend_from_past(self, user_transactions, n=10) -> List[str]:
        """
        Return recommendations based on past user transactions.
        """
        rec = []
        for transaction in user_transactions:
            rec.extend(self.recommend_from_single(transaction['product_id']))
        random.shuffle(rec)
        return rec[:n]

In [10]:

class SimilutudeRecommender(RecommendationAbstract):
    """
    Features trainning using KNN model but purely recommendations are purely based on cosine similarity.
    Supports Extending. Recommnedations (that can be used by other Recommender classes) to find neighbors.
    """
    strategy_name: str = "Similitude Recommender"
    slug_name: str = "simmilitude_recommender"
    version: str = "v1"
    details: str = "REQUIRES IMPLEMENTATION"
    link: str = "REQUIRES IMPLEMENTATION"
    supports_single_recommendation: bool = True
    supports_past_recommendation: bool = True
    
    def __init__(self, products: pd.DataFrame, product_data: dict, transactions = None):
        super().__init__(products, product_data)
        self.products = products
        self.model = None
        
        # Get the product ids and store them.
        self.product_ids = self.products['id'].unique()
        self.all_transactions_df = transactions
        
    def train(self, transactions, auto_save=True, dont_save_self_state=False) :
        
        sim_options = {"name": "pearson", "user_based": False}
        model = KNNBasic(sim_options=sim_options)
        
        reader = Reader(rating_scale=(1, 5))
        
        data = Dataset.load_from_df(transactions[['user_id', 'product_id', 'rate']], reader)
        
        model.fit(data.build_full_trainset())
        
        if dont_save_self_state:
            return model
        
        self.model = model
        self.all_transactions_df = transactions
        
        if auto_save:
            self.save()
            
        return model
        
        
    def get_filename(self):
        return "models/" + self.slug_name + self.product_data["unique_name"] + ".pik"
    
    def save(self):
        # Store self.pt
        filename = self.get_filename()
        model_file = open(filename, 'wb')
        pickle.dump(self.model, model_file)
        model_file.close()
        
    def load(self, auto_create=True):
        
        filename = self.get_filename()
        try:
            model_file = open(filename, 'rb')
            self.model = pickle.load(model_file)
            model_file.close()
        except:
            self.save()
            
    def getNeighbors(self, product_id: str, n=5):
        """
        Returns the neighbors of a product
        """
        product_inner_id = self.model.trainset.to_inner_iid(product_id)
        neighbors = self.model.get_neighbors(product_inner_id, k=n)
        return neighbors

    def recommend_from_single(self, product_id: str, n=5) -> List[Tuple[dict, float]]:
        """
        
        # To optimize things, SVD takes a Similitude type recommender. Which posses the method (receive product neighbors.)
        """
        recommendation_list: List[tuple[dict, float]] = []
        product_inner_id = self.model.trainset.to_inner_iid(product_id)
        neighbors = self.model.get_neighbors(product_inner_id, k=n)
        
        # for each neighbor, try to predict and prioritize given a user in all_transactions_that shared that book as well.
        for neighbor_book_inner_id in neighbors:
            # get user_id that top rated the product sort the relevant_transactions
            if neighbor_book_inner_id == product_inner_id:
                continue
            
            product_serie = self.products.iloc[neighbor_book_inner_id]
            neighbor_book_id = product_serie['id']
            product = self.id_to_products[neighbor_book_id]
            
            if product['product_id'] == product_id:
                continue
            
            recommendation_list.append((self.id_to_products[neighbor_book_id], 1))
        
        # sort recommendations
        random.shuffle(recommendation_list)
        return recommendation_list[:n]
        

    def recommend_from_past(self, transactions: List[str], n=10):
        """
        Calls for each transaction the recommend_from_single method.
        Gives Priority if seen multiple recommendations.
        Shuffle and returns :n
        """
        recs = set()
        recs_seen_times = {}
        products_dictionary = {}
        
        # Deprecated.
        # if(len(transactions) > 2):
        #     return self.collaborativestore_predict_population(
        #         transactions, n=n
        #     )
        
        for transaction in transactions:
            recs = self.recommend_from_single(transaction)
            for rec_id, confidence in recs:
                
                if rec_id in recs:
                    recs_seen_times[rec_id['id']] += 1
                else:
                    products_dictionary[rec_id['id']] = rec_id
                    recs_seen_times[rec_id['id']] = 1
        
        for rec_id in recs_seen_times:
            recs.append((products_dictionary[rec_id], recs_seen_times[rec_id]))
            
        recs = list(recs)
        
        recs.sort(key=lambda x: x[1], reverse=True)
        return recs
    

In [25]:

class MatrixRecommender(RecommendationAbstract):
    strategy_name: str = "Matrix Basic"
    slug_name: str = "matrix"
    version: str = "v1"
    details: str = "REQUIRES IMPLEMENTATION"
    link: str = "REQUIRES IMPLEMENTATION"
    supports_single_recommendation: bool = True
    supports_past_recommendation: bool = True
    algorithm=SVD
    
    def __init__(self, products: pd.DataFrame, product_data: dict, similitudeRec=SimilutudeRecommender, transactions = None):
        super().__init__(products, product_data)
        self.products = products
        self.model = None
        
        # Get the product ids and store them.
        self.product_ids = self.products['id'].unique()
        self.all_transactions_df = transactions
        self.similitudeRec = similitudeRec(products, product_data, transactions)
        
    def train(self, transactions, auto_save=True, dont_save_self_state=False) :
        self.similitudeRec.train(transactions)
                
        # sim_options = {"name": "pearson_baseline", "user_based": False}
        model = self.algorithm()
        
        reader = Reader(rating_scale=(1, 5))
        
        data = Dataset.load_from_df(transactions[['user_id', 'product_id', 'rate']], reader)
        
        model.fit(data.build_full_trainset())
        
        if dont_save_self_state:
            return model
        
        self.model = model
        self.all_transactions_df = transactions
        # self.accuracy = accuracy.rmse(model.test(data.build_full_trainset().build_testset()), verbose=True)
        
        if auto_save:
            self.save()
            
        return model
        
        
    def get_filename(self):
        return "models/" + self.slug_name + self.product_data["unique_name"] + ".pik"
    
    def save(self):
        # Store self.pt
        self.similitudeRec.save()
        filename = self.get_filename()
        model_file = open(filename, 'wb')
        pickle.dump(self.model, model_file)
        model_file.close()
        
    def load(self, auto_create=True):
        self.similitudeRec.load(auto_create=auto_create)
        filename = self.get_filename()
        model_file = open(filename, 'rb')
        self.model = pickle.load(model_file)
        model_file.close()

    def recommend_from_single(self, product_id: str, n=5) -> List[Tuple[dict, float]]:
        """
        
        # To optimize things, SVD takes a Similitude type recommender. Which posses the method (receive product neighbors.)
        """
        recommendation_list: List[tuple[dict, float]] = []
        # neighbors = self.model.get_neighbors(product_inner_id, k=n*2)
        neighbors = self.similitudeRec.getNeighbors(product_id, n=n*2)
        
        # for each neighbor, try to predict and prioritize given a user in all_transactions_that shared that book as well.
        for neighbor_book_inner_id in neighbors:
            
            product_serie = self.products.iloc[neighbor_book_inner_id]
            neighbor_book_id = product_serie['id']
            
            if (neighbor_book_id == product_id):
                continue
            
            relevant_transactions = self.all_transactions_df[self.all_transactions_df['product_id'] == neighbor_book_id]
            relevant_transactions = relevant_transactions.sort_values(by='rate', ascending=False)
            # remove where  product_id product_id
            if len(relevant_transactions) == 0:
                continue
            
            
            user_id = relevant_transactions.iloc[0]['user_id']
            
            pred = self.model.predict(user_id, neighbor_book_id)
            recommendation_list.append((self.id_to_products[neighbor_book_id], pred.est))
        
        # sort recommendations
        recommendation_list.sort(key=lambda x: x[1], reverse=True)
        return recommendation_list[:n]

    def collaborativestore_predict_population(self, transactions: List[str], n=5):
        """
        Adds the transactions to the use history to be considered when training the model. Doesnt not save the model with this transactions,
        proceeds to use the models to create recommendations. This is pattern was added for KNN and Matrix Factorizations
        """
        # Add transactions to the self.transactions_df as a new user
        transaction_rows = []
        random_user_id = "user" + str(random.randint(0, 1000000))
        for transaction in transactions:
            transaction_rows.append({'user_id': 'user_id', 'product_id': transaction, 'rate': 5})
        
        # Convert to a DataFrame
        new_transactions_df = pd.DataFrame(transaction_rows)

        # Append using concat
        all_transactions_df: pd.Dataframe = pd.concat([self.all_transactions_df, new_transactions_df], ignore_index=True)
        
        model = self.train(all_transactions_df, dont_save_self_state=True)
        
        return self.predict_recommendations(random_user_id, transactions, model, n)
    
    def predict_recommendations(self, user_id: str, transactions: List[str], model, n=5):
        books_to_predict = [book_id for book_id in self.product_ids if book_id not in transactions]
        predictions = []
        
        for book_id in books_to_predict:
            pred = model.predict(user_id, book_id)
            predictions.append((book_id, pred.est))
        
        pred_products = []
        # sort predictions
        predictions.sort(key=lambda x: x[1], reverse=True)
        for book_id, confidence in predictions[:n]:
            product = self.id_to_products[book_id]
            pred_products.append(product)
            
        return pred_products
        

    def recommend_from_past(self, transactions: List[str], n=10):
        """
        Calls for each transaction the recommend_from_single method.
        Gives Priority if seen multiple recommendations.
        Shuffle and returns :n
        """
        recs = set()
        recs_seen_times = {}
        products_dictionary = {}
        
        for transaction in transactions:
            recs = self.recommend_from_single(transaction)
            for rec_id, confidence in recs:
                
                if rec_id in recs:
                    recs_seen_times[rec_id['id']] += 1
                else:
                    products_dictionary[rec_id['id']] = rec_id
                    recs_seen_times[rec_id['id']] = 1
        
        for rec_id in recs_seen_times:
            recs.append((products_dictionary[rec_id], recs_seen_times[rec_id]))
            
        recs = list(recs)
        
        recs.sort(key=lambda x: x[1], reverse=True)
        return recs
    

In [26]:

engineRec = MatrixRecommender(productdf, product_data, transactions=transactiondf)
engineRec.train(transactions=transactiondf, auto_save=True)
# engineRec.load()
  
  

randomProduct = engineRec.get_random_recommendation()[0]
pprint.pprint(randomProduct)
SEARCH_TEST = randomProduct['id']
# SEARCH_TEST = engineRec.recommend_from_single(randomProduct['id'])
# 

# pprint.pprint(engineRec.id_to_productDetail(SEARCH_TEST))
# pprint.pprint(SEARCH_TEST)
print('======== RECOMENDATIONS SINGLE CASE =========== ')

# SEARCH_TEST = '1572971835'
rec = engineRec.recommend_from_single(SEARCH_TEST)
print('---------')
pprint.pprint(rec)

SEARCH_TEST = '0590353403'
rec = engineRec.recommend_from_single(SEARCH_TEST)
print('======== RECOMENDATIONS SINGLE CASE =========== ')

pprint.pprint(engineRec.id_to_productDetail(SEARCH_TEST))
print('---------')
pprint.pprint(rec)


# ... Repetition.
print("=============  RECOMENDATIONS RECOMMENDATIONS  ============")
tansactions = ['0590353403', '0439139597']

"""
Harry Potter and the Sorcerer's Stone (Book 1)
"Harry Potter and the Goblet of Fire (Book 4)"
"""

rec = engineRec.recommend_from_past(tansactions)
pprint.pprint(rec)



Computing the pearson similarity matrix...
Done computing similarity matrix.
{'count': 22,
 'id': '0061054909',
 'product_id': '0061054909',
 'product_image': 'http://images.amazon.com/images/P/0061054909.01.MZZZZZZZ.jpg',
 'product_price': nan,
 'product_soup': 'Love in Vein Poppy Z. Brite Eos',
 'product_tags': nan,
 'product_title': 'Love in Vein'}
======== RECOMENDATIONS SINGLE CASE =========== 
---------
[({'count': 163,
   'id': '0451166892',
   'product_id': '0451166892',
   'product_image': 'http://images.amazon.com/images/P/0451166892.01.MZZZZZZZ.jpg',
   'product_price': nan,
   'product_soup': 'The Pillars of the Earth Ken Follett Signet Book',
   'product_tags': nan,
   'product_title': 'The Pillars of the Earth'},
  5),
 ({'count': 164,
   'id': '0345339703',
   'product_id': '0345339703',
   'product_image': 'http://images.amazon.com/images/P/0345339703.01.MZZZZZZZ.jpg',
   'product_price': nan,
   'product_soup': 'The Fellowship of the Ring (The Lord of the Rings, Part 1

In [34]:

class SVDMatrixRecommender(MatrixRecommender):
    strategy_name: str = "SVD Factorization"
    slug_name: str = "svd_recommender"
    version: str = "v1"
    details: str = "REQUIRES IMPLEMENTATION"
    link: str = "REQUIRES IMPLEMENTATION"
    supports_single_recommendation: bool = True
    supports_past_recommendation: bool = True
    algorithm=SVD

In [32]:

class SVDPPMatrixRecommender(MatrixRecommender):
    strategy_name: str = "SVD PP Matrix Factorization"
    slug_name: str = "svdpp_recommender"
    version: str = "v1"
    details: str = "REQUIRES IMPLEMENTATION"
    link: str = "REQUIRES IMPLEMENTATION"
    supports_single_recommendation: bool = True
    supports_past_recommendation: bool = True
    algorithm=SVDpp
    

In [33]:

engineRec = SVDPPMatrixRecommender(productdf, product_data, transactions=transactiondf)
engineRec.train(transactions=transactiondf, auto_save=True)
# engineRec.load()
  
  

randomProduct = engineRec.get_random_recommendation()[0]
pprint.pprint(randomProduct)
SEARCH_TEST = randomProduct['id']
# SEARCH_TEST = engineRec.recommend_from_single(randomProduct['id'])
# 

# pprint.pprint(engineRec.id_to_productDetail(SEARCH_TEST))
# pprint.pprint(SEARCH_TEST)
print('======== RECOMENDATIONS SINGLE CASE =========== ')

# SEARCH_TEST = '1572971835'
rec = engineRec.recommend_from_single(SEARCH_TEST)
print('---------')
pprint.pprint(rec)

SEARCH_TEST = '0590353403'
rec = engineRec.recommend_from_single(SEARCH_TEST)
print('======== RECOMENDATIONS SINGLE CASE =========== ')

pprint.pprint(engineRec.id_to_productDetail(SEARCH_TEST))
print('---------')
pprint.pprint(rec)


# ... Repetition.
print("=============  RECOMENDATIONS RECOMMENDATIONS  ============")
tansactions = ['0590353403', '0439139597']

"""
Harry Potter and the Sorcerer's Stone (Book 1)
"Harry Potter and the Goblet of Fire (Book 4)"
"""

rec = engineRec.recommend_from_past(tansactions)
pprint.pprint(rec)



Computing the pearson similarity matrix...
Done computing similarity matrix.
{'count': 27,
 'id': '0451193288',
 'product_id': '0451193288',
 'product_image': 'http://images.amazon.com/images/P/0451193288.01.MZZZZZZZ.jpg',
 'product_price': nan,
 'product_soup': 'Reckless Endangerment Robert K. Tanenbaum Signet Book',
 'product_tags': nan,
 'product_title': 'Reckless Endangerment'}
======== RECOMENDATIONS SINGLE CASE =========== 
---------
[({'count': 129,
   'id': '0385336179',
   'product_id': '0385336179',
   'product_image': 'http://images.amazon.com/images/P/0385336179.01.MZZZZZZZ.jpg',
   'product_price': nan,
   'product_soup': 'Shopaholic Ties the Knot Sophie Kinsella Delta',
   'product_tags': nan,
   'product_title': 'Shopaholic Ties the Knot'},
  4.71529792390149),
 ({'count': 110,
   'id': '0312983824',
   'product_id': '0312983824',
   'product_image': 'http://images.amazon.com/images/P/0312983824.01.MZZZZZZZ.jpg',
   'product_price': nan,
   'product_soup': "Faking It Jen

In [40]:

class NMFMatrixRecommender(MatrixRecommender):
    strategy_name: str = "NMF Matrix Factorization"
    slug_name: str = "nmf_matrix_factorization"
    version: str = "v1"
    details: str = "REQUIRES IMPLEMENTATION"
    link: str = "REQUIRES IMPLEMENTATION"
    supports_single_recommendation: bool = True
    supports_past_recommendation: bool = True
    algorithm=NMF
    

In [41]:

engineRec = NMFMatrixRecommender(productdf, product_data, transactions=transactiondf)
engineRec.train(transactions=transactiondf, auto_save=True)
# engineRec.load()
  
  

randomProduct = engineRec.get_random_recommendation()[0]
pprint.pprint(randomProduct)
SEARCH_TEST = randomProduct['id']
print('======== RECOMENDATIONS SINGLE CASE =========== ')

# SEARCH_TEST = '1572971835'
rec = engineRec.recommend_from_single(SEARCH_TEST)
print('---------')
pprint.pprint(rec)

SEARCH_TEST = '0590353403'
rec = engineRec.recommend_from_single(SEARCH_TEST)
print('======== RECOMENDATIONS SINGLE CASE =========== ')

pprint.pprint(engineRec.id_to_productDetail(SEARCH_TEST))
print('---------')
pprint.pprint(rec)


# ... Repetition.
print("=============  RECOMENDATIONS RECOMMENDATIONS  ============")
tansactions = ['0590353403', '0439139597']

"""
Harry Potter and the Sorcerer's Stone (Book 1)
"Harry Potter and the Goblet of Fire (Book 4)"
"""

rec = engineRec.recommend_from_past(tansactions)
pprint.pprint(rec)



Computing the pearson similarity matrix...
Done computing similarity matrix.
{'count': 14,
 'id': '0451192575',
 'product_id': '0451192575',
 'product_image': 'http://images.amazon.com/images/P/0451192575.01.MZZZZZZZ.jpg',
 'product_price': nan,
 'product_soup': 'Donnie Brasco: My Undercover Life in the Mafia Joseph D. '
                 'Pistone Signet Book',
 'product_tags': nan,
 'product_title': 'Donnie Brasco: My Undercover Life in the Mafia'}
======== RECOMENDATIONS SINGLE CASE =========== 
---------
[({'count': 65,
   'id': '0446600474',
   'product_id': '0446600474',
   'product_image': 'http://images.amazon.com/images/P/0446600474.01.MZZZZZZZ.jpg',
   'product_price': nan,
   'product_soup': 'Season of the Machete James Patterson Warner Books',
   'product_tags': nan,
   'product_title': 'Season of the Machete'},
  4.122461025070084),
 ({'count': 105,
   'id': '031242227X',
   'product_id': '031242227X',
   'product_image': 'http://images.amazon.com/images/P/031242227X.01.MZZZ

In [42]:

from surprise.prediction_algorithms.slope_one import SlopeOne

class SlopeOneRecommender(MatrixRecommender):
    strategy_name: str = "Slope One Recommender"
    slug_name: str = "slope_recommender"
    version: str = "v1"
    details: str = "REQUIRES IMPLEMENTATION"
    link: str = "REQUIRES IMPLEMENTATION"
    supports_single_recommendation: bool = True
    supports_past_recommendation: bool = True
    algorithm=SlopeOne
    

In [43]:

engineRec = SlopeOneRecommender(productdf, product_data, transactions=transactiondf)
engineRec.train(transactions=transactiondf, auto_save=True)
# engineRec.load()
  
  

randomProduct = engineRec.get_random_recommendation()[0]
pprint.pprint(randomProduct)
SEARCH_TEST = randomProduct['id']
# SEARCH_TEST = engineRec.recommend_from_single(randomProduct['id'])
# 

# pprint.pprint(engineRec.id_to_productDetail(SEARCH_TEST))
# pprint.pprint(SEARCH_TEST)
print('======== RECOMENDATIONS SINGLE CASE =========== ')

# SEARCH_TEST = '1572971835'
rec = engineRec.recommend_from_single(SEARCH_TEST)
print('---------')
pprint.pprint(rec)

SEARCH_TEST = '0590353403'
rec = engineRec.recommend_from_single(SEARCH_TEST)
print('======== RECOMENDATIONS SINGLE CASE =========== ')

pprint.pprint(engineRec.id_to_productDetail(SEARCH_TEST))
print('---------')
pprint.pprint(rec)


# ... Repetition.
print("=============  RECOMENDATIONS RECOMMENDATIONS  ============")
tansactions = ['0590353403', '0439139597']

"""
Harry Potter and the Sorcerer's Stone (Book 1)
"Harry Potter and the Goblet of Fire (Book 4)"
"""

rec = engineRec.recommend_from_past(tansactions)
pprint.pprint(rec)



Computing the pearson similarity matrix...
Done computing similarity matrix.
{'count': 19,
 'id': '0375401091',
 'product_id': '0375401091',
 'product_image': 'http://images.amazon.com/images/P/0375401091.01.MZZZZZZZ.jpg',
 'product_price': nan,
 'product_soup': 'A Certain Justice (Adam Dalgliesh Mysteries (Hardcover)) P. '
                 'D. James Alfred A. Knopf',
 'product_tags': nan,
 'product_title': 'A Certain Justice (Adam Dalgliesh Mysteries (Hardcover))'}
======== RECOMENDATIONS SINGLE CASE =========== 
---------
[({'count': 63,
   'id': '0679442790',
   'product_id': '0679442790',
   'product_image': 'http://images.amazon.com/images/P/0679442790.01.MZZZZZZZ.jpg',
   'product_price': nan,
   'product_soup': 'The Reader Bernhard Schlink Random House',
   'product_tags': nan,
   'product_title': 'The Reader'},
  4.10233760170735),
 ({'count': 210,
   'id': '0380789035',
   'product_id': '0380789035',
   'product_image': 'http://images.amazon.com/images/P/0380789035.01.MZZZZZZZ

In [45]:

from surprise.prediction_algorithms.co_clustering import CoClustering

class CoClusteringRecommender(MatrixRecommender):
    strategy_name: str = "Slope One Recommender"
    slug_name: str = "slope_recommender"
    version: str = "v1"
    details: str = "REQUIRES IMPLEMENTATION"
    link: str = "REQUIRES IMPLEMENTATION"
    supports_single_recommendation: bool = True
    supports_past_recommendation: bool = True
    algorithm=CoClustering
    

In [46]:

engineRec = CoClusteringRecommender(productdf, product_data, transactions=transactiondf)
engineRec.train(transactions=transactiondf, auto_save=True)
# engineRec.load()
  
  

randomProduct = engineRec.get_random_recommendation()[0]
pprint.pprint(randomProduct)
SEARCH_TEST = randomProduct['id']
# SEARCH_TEST = engineRec.recommend_from_single(randomProduct['id'])
# 

# pprint.pprint(engineRec.id_to_productDetail(SEARCH_TEST))
# pprint.pprint(SEARCH_TEST)
print('======== RECOMENDATIONS SINGLE CASE =========== ')

# SEARCH_TEST = '1572971835'
rec = engineRec.recommend_from_single(SEARCH_TEST)
print('---------')
pprint.pprint(rec)

SEARCH_TEST = '0590353403'
rec = engineRec.recommend_from_single(SEARCH_TEST)
print('======== RECOMENDATIONS SINGLE CASE =========== ')

pprint.pprint(engineRec.id_to_productDetail(SEARCH_TEST))
print('---------')
pprint.pprint(rec)


# ... Repetition.
print("=============  RECOMENDATIONS RECOMMENDATIONS  ============")
tansactions = ['0590353403', '0439139597']

"""
Harry Potter and the Sorcerer's Stone (Book 1)
"Harry Potter and the Goblet of Fire (Book 4)"
"""

rec = engineRec.recommend_from_past(tansactions)
pprint.pprint(rec)

